In [5]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
import scipy.io
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def load_dataset(path, split=0.2):
    train_x = sorted(glob(os.path.join(path, "images1", "*")))
    train_y = sorted(glob(os.path.join(path, "masks1", "*")))


    split_size = int(split * len(train_x))



    train_x, valid_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)



def read_image_mask(x, y):
    """ Reading """
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    y = cv2.imread(y, cv2.IMREAD_COLOR)
    assert x.shape == y.shape

    """ Resizing """
    x = cv2.resize(x, (IMG_W, IMG_H))
    y = cv2.resize(y, (IMG_W, IMG_H))

    """ Image processing """
    x = x / 255.0
    x = x.astype(np.float32)

    """ Mask processing """
    output = []
    for color in COLORMAP:
        cmap = np.all(np.equal(y, color), axis=-1)
        output.append(cmap)
    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return x, output

def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
        image, mask = read_image_mask(x, y)
        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    IMG_H = 512
    IMG_W = 512
    NUM_CLASSES = 4
    input_shape = (IMG_H, IMG_W, 3)

    batch_size = 16
    lr = 1e-4
    num_epochs = 35


    # dataset_path = "C:\\Users\\asus\\Downloads\\seg_data_new"
    dataset_path ="C:\\Users\\asus\\Downloads\\resized_train_data"
    save_image_path = "C:\\Users\\asus\\Downloads\\resized_train_data\\new_masks"


    """ Loading the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
    print("")

    """ Process the colormap """
    # CLASSES= ['low-high-medium','2_Fair_Steel_Corrosion','3_Poor_Steel_Corrosion','4_Severe_Steel_Corrosion']
    COLORMAP =[ [0,0,0] , [138, 192, 208], [21, 179, 42], [189, 148, 86] ]


    # color_map ={"BackGround": (0, 0, 0), "Fair": (138, 192, 208), "Poor": (21, 179, 42), "Severe": (189, 148, 86)}

    """ Dataset Pipeline """
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)



Train: 459/459 - Valid: 152/152 - Test: 152/152



In [25]:
# import os
# import cv2
# import pickle
# import numpy as np

# from random import random




# class CustomDataset():
#     def __init__(self, folder_path, transform=None):
#         self.folder_path = folder_path
#         self.image_files = sorted(glob(os.path.join(folder_path, 'images', '*')))
#         self.mask_files = sorted(glob(os.path.join(folder_path, 'masks', '*')))
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         image_path = self.image_files[idx]
#         mask_path = self.mask_files[idx]

#         image = cv2.imread(image_path)
#         mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

#         sample = {'image': image, 'mask': mask}

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

# # Example usage
# folder_path = "C:\\Users\\asus\\Downloads\\paper_512"
# dataset = CustomDataset(folder_path)



In [ ]:
# import matplotlib.pyplot as plt
# sample = dataset[0]
# plt.figure(figsize=(10, 5))
# plt.subplot(1,2,1)
# plt.imshow(sample['image'],cmap='plasma')
# plt.axis('off')


# plt.subplot(1,2,2)
# plt.imshow(sample['mask'])
# plt.axis('off')


In [8]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



def grayscale_to_rgb(mask, classes, colormap):
    h, w, _ = mask.shape
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(colormap[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

def save_results(image, mask, pred, save_image_path):
    h, w, _ = image.shape
    line = np.ones((h, 10, 3)) * 255

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, CLASSES, COLORMAP)

    cat_images = np.concatenate([image, line, mask, line, pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)



    """ Load the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")
    print("")

    """ Evaluation and Prediction """
    i=0
    SCORE = []
    for x, y in tqdm(zip(valid_x, valid_y), total=len(valid_x)):
        """ Extract the name """
        name = x.split("/")[-1].split(".")[0]
        name = name[41:]
        print(name,"hii")

        """ Reading the image """
        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMG_W, IMG_H))
        image_x = image
        image = image/255.0
        image = np.expand_dims(image, axis=0)

        """ Reading the mask """
        print(x,y)
        mask = cv2.imread(y, cv2.IMREAD_COLOR)
        mask = cv2.resize(mask, (IMG_W, IMG_H))
        mask_x = mask
        onehot_mask = []
        for color in COLORMAP:
            # new_color = [color[2],color[1],color[0]]
            cmap = np.all(np.equal(mask, color), axis=-1)
            onehot_mask.append(cmap)
        onehot_mask = np.stack(onehot_mask, axis=-1)
        onehot_mask = np.argmax(onehot_mask, axis=-1)
        onehot_mask = onehot_mask.astype(np.int32)
        save_image = f"{save_image_path}\{name}.png"
        print(save_image)
        # print(onehot_mask.shape,"hii")
        # plt.figure(figsize=(10, 5))
        # plt.subplot(1,3,1)
        # plt.imshow(onehot_mask)
        # plt.subplot(1,3,2)
        # plt.imshow(image[0])
        # plt.subplot(1,3,3)
        # plt.imshow(mask)
        # plt.axis('off')
        cv2.imwrite(save_image,onehot_mask)

        # """ Prediction """
        # pred = model.predict(image, verbose=0)[0]
        # pred = np.argmax(pred, axis=-1)
        # pred = pred.astype(np.float32)

        # """ Saving the prediction """

        # save_results(image_x, mask_x, pred, save_image_path)

    #     """ Flatten the array """
    #     onehot_mask = onehot_mask.flatten()
    #     pred = pred.flatten()

    #     labels = [i for i in range(NUM_CLASSES)]

    #     """ Calculating the metrics values """
    #     f1_value = f1_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
    #     jac_value = jaccard_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)

    #     SCORE.append([f1_value, jac_value])

    # """ Metrics values """
    # score = np.array(SCORE)
    # score = np.mean(score, axis=0)

    # f = open("/content/drive/MyDrive/files/data.csv", "w")
    # f.write("Class,F1,Jaccard\n")

    # l = ["Class", "F1", "Jaccard"]
    # print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
    # print("-"*35)

    # for i in range(score.shape[1]):
    #     class_name = CLASSES[i]
    #     f1 = score[0, i]
    #     jac = score[1, i]
    #     dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    #     print(dstr)
    #     f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # print("-"*35)
    # class_mean = np.mean(score, axis=-1)
    # class_name = "Mean"
    # f1 = class_mean[0]
    # jac = class_mean[1]
    # dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    # print(dstr)
    # f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # f.close()

Train: 459/459 - Valid: 152/152 - Test: 152/152



  0%|          | 0/459 [00:00<?, ?it/s]

  0%|          | 2/459 [00:00<00:40, 11.32it/s]

a\images1\image_270 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_270.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_270.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_270.png
a\images1\image_360 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_360.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_360.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_360.png
a\images1\image_154 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_154.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_154.png


  1%|          | 4/459 [00:00<00:47,  9.64it/s]

...

C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_212.png
a\images1\image_145 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_145.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_145.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_145.png
a\images1\image_268 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_268.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_268.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_268.png
a\images1\image_89 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_89.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_89.png


 72%|███████▏  | 331/459 [00:27<00:10, 12.08it/s]

C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_89.png
a\images1\image_599 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_599.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_599.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_599.png
a\images1\image_217 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_217.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_217.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_217.png
a\images1\image_680 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_680.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_680.png


 73%|███████▎  | 335/459 [00:27<00:09, 12.42it/s]

C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_680.png
a\images1\image_97 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_97.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_97.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_97.png
a\images1\image_720 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_720.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_720.png
C:\Users\asus\Downloads\resized_train_data\new_masks\a\images1\image_720.png
a\images1\image_334 hii
C:\Users\asus\Downloads\resized_train_data\images1\image_334.jpg C:\Users\asus\Downloads\resized_train_data\masks1\mask_334.png
